In [1]:
import gymnasium as gym
import numpy as np
import math

In [6]:
is_training = False
render = True

episodes = 5
learning_rate_a = 0.95
discount_factor_g = 0.98

epsilon = 1
epsilon_decay_rate = 2 / episodes
rng = np.random.default_rng()

In [7]:
env = gym.make('FrozenLake-v1', desc=['SFHH','FFFF','HHFF','GFFH'], map_name="4x4", is_slippery=False, render_mode='human' if render else None)
goal = 12

In [4]:
q = np.zeros((env.observation_space.n,env.action_space.n))

In [8]:
for i in range(episodes):
    terminated = False
    steps = 0
    
    state = env.reset()[0]

    while(not terminated and steps < 30):
        if is_training and rng.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q[state, :])

        new_state,reward,terminated,_,_ = env.step(action)
        if terminated and new_state != goal:
            reward = -2
        elif not terminated:
            sx = (new_state % 4) + 1
            sy = math.floor(new_state/4)+1
            gx = (goal % 4) + 1
            gy = math.floor(goal/4)+1
            reward = ((sx+gx)/2+(sy+gy)/2)*1e-5

        if is_training:
            q[state, action] = q[state, action]+learning_rate_a*(
                reward+discount_factor_g*np.max(q[new_state,:])-q[state, action]
            )

        steps += 1

        if reward > 0 and terminated: 
            print(f'Episode {i+1} won in {steps} steps')
        elif terminated: 
            print(f'Episode {i+1} lost  in {steps} steps')

        state = new_state
    epsilon = max(epsilon - epsilon_decay_rate, 0)
env.close()

Episode 1 won in 7 steps
Episode 2 won in 7 steps
Episode 3 won in 7 steps
Episode 4 won in 7 steps
Episode 5 won in 7 steps
